## Day 49 Lecture 1 Assignment

In this assignment, we will apply GMM (Gaussian Mixture Modeling) clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.special import entr

<IPython.core.display.Javascript object>

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [3]:
# answer goes here

nba_df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/nba_player_seasons.csv')
nba_df


,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2019,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,...,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,2019,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,...,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,2019,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,...,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,...,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,...,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2136,2016,Joe Young,PG,23,IND,41,0,9.4,1.5,4.1,...,0.800,0.1,1.1,1.2,1.6,0.4,0.0,0.8,0.7,3.8
2137,2016,Nick Young,SG,30,LAL,54,2,19.1,2.3,6.9,...,0.829,0.3,1.5,1.8,0.6,0.4,0.1,0.6,0.9,7.3
2138,2016,Thaddeus Young,PF,27,BRK,73,73,33.0,6.8,13.2,...,0.644,2.4,6.6,9.0,1.8,1.5,0.5,1.9,2.5,15.1
2139,2016,Cody Zeller,C,23,CHO,73,60,24.3,3.2,6.0,...,0.754,1.9,4.3,6.2,1.0,0.8,0.9,0.9,2.8,8.7


<IPython.core.display.Javascript object>

The goal is to cluster these player-seasons to identify potential player "archetypes".  
The pre-processing steps will be identical to what we previously did for K-means.

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

In [4]:
# answer goes here

nba_df2 = nba_df.loc[lambda x: (x.GS >=20) & (x.MP >= 10)].copy()
nba_df2


,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,...,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,...,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
7,2019,LaMarcus Aldridge,C,33,SAS,81,81,33.2,8.4,16.3,...,0.847,3.1,6.1,9.2,2.4,0.5,1.3,1.8,2.2,21.3
10,2019,Jarrett Allen,C,20,BRK,80,80,26.2,4.2,7.1,...,0.709,2.4,6.0,8.4,1.4,0.5,1.5,1.3,2.3,10.9
12,2019,Al-Farouq Aminu,PF,28,POR,81,81,28.3,3.2,7.3,...,0.867,1.4,6.1,7.5,1.3,0.8,0.4,0.9,1.8,9.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2122,2016,Deron Williams,PG,31,DAL,65,63,32.4,4.9,11.9,...,0.869,0.3,2.6,2.9,5.8,0.9,0.2,2.3,2.4,14.1
2125,2016,Lou Williams,SG,29,LAL,67,35,28.5,4.2,10.3,...,0.830,0.5,2.1,2.5,2.5,0.9,0.3,1.6,1.6,15.3
2126,2016,Marvin Williams,PF,29,CHO,81,81,28.9,4.2,9.2,...,0.833,1.6,4.9,6.4,1.4,0.7,1.0,0.8,1.6,11.7
2138,2016,Thaddeus Young,PF,27,BRK,73,73,33.0,6.8,13.2,...,0.644,2.4,6.6,9.0,1.8,1.5,0.5,1.9,2.5,15.1


<IPython.core.display.Javascript object>

Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [5]:
# answer goes here

X = nba_df2[["PTS", "TRB", "AST", "STL", "BLK", "TOV", "3P"]]
X

,PTS,TRB,AST,STL,BLK,TOV,3P
3,13.9,9.5,1.6,1.5,1.0,1.7,0.0
4,8.9,7.3,2.2,0.9,0.8,1.5,0.0
7,21.3,9.2,2.4,0.5,1.3,1.8,0.1
10,10.9,8.4,1.4,0.5,1.5,1.3,0.1
12,9.4,7.5,1.3,0.8,0.4,0.9,1.2
...,...,...,...,...,...,...,...
2122,14.1,2.9,5.8,0.9,0.2,2.3,1.5
2125,15.3,2.5,2.5,0.9,0.3,1.6,1.6
2126,11.7,6.4,1.4,0.7,1.0,0.8,1.9
2138,15.1,9.0,1.8,1.5,0.5,1.9,0.1


<IPython.core.display.Javascript object>

Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [6]:
# answer goes here
scale = StandardScaler()

X_scale = pd.DataFrame(scale.fit_transform(X), columns=X.columns)
X_scale

,PTS,TRB,AST,STL,BLK,TOV,3P
0,0.114638,1.612909,-0.617816,1.393220,0.824001,0.023511,-1.353529
1,-0.744985,0.768627,-0.330886,-0.050271,0.420162,-0.218831,-1.353529
2,1.386880,1.497780,-0.235242,-1.012598,1.429758,0.144681,-1.240535
3,-0.401136,1.190768,-0.713459,-1.012598,1.833596,-0.461172,-1.240535
4,-0.659023,0.845380,-0.761281,-0.290853,-0.387514,-0.945854,0.002389
...,...,...,...,...,...,...,...
799,0.149023,-0.919937,1.390695,-0.050271,-0.791352,0.750535,0.341369
800,0.355332,-1.073443,-0.187421,-0.050271,-0.589433,-0.097660,0.454362
801,-0.263596,0.423239,-0.713459,-0.531435,0.824001,-1.067025,0.793341
802,0.320947,1.421027,-0.522172,1.393220,-0.185595,0.265852,-1.240535


<IPython.core.display.Javascript object>

Run both K-Means and Gaussian mixtures modeling twice, once with 3 cluster and once with 7 clusters. Print out the resulting means for all 4 scenarios (KM+3, GMM+3, KM+7, GMM+7). When printing the means, transform the scaled versions back into their corresponding unscaled values. 

What "archetypes" do you see? Are the archetypes identified by GMM similar to those identified by K-Means? How do the means of GMM differ from those of K-Means?

In [7]:
# answer goes here
# KM3
kmeans = KMeans(n_clusters=3)
kmeans.fit(X_scale)
centers = kmeans.cluster_centers_
centers_unscaled = pd.DataFrame(scale.inverse_transform(centers), columns=X.columns)
centers_unscaled.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV,3P
0,19.262944,5.253807,5.670558,1.342640,0.502030,2.701523,1.816244
1,10.421667,3.914524,2.046190,0.799048,0.358810,1.214762,1.249524
2,13.195722,8.448128,1.864171,0.750267,1.210160,1.651337,0.430481


<IPython.core.display.Javascript object>

In [8]:
# GM3
gmm = GaussianMixture(n_components=3)
gmm.fit(X_scale)
means = gmm.means_
means_unscaled = pd.DataFrame(scale.inverse_transform(means), columns=X.columns)
means_unscaled.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV,3P
0,16.058030,4.281715,4.914087,1.200884,0.377321,2.281709,1.605860
1,14.248222,8.421910,2.217434,0.780511,1.233776,1.811043,0.527970
2,10.366509,4.385985,1.619330,0.770843,0.409657,1.118171,1.238169


<IPython.core.display.Javascript object>

In [9]:
# KM7
kmeans = KMeans(n_clusters=7)
kmeans.fit(X_scale)
centers = kmeans.cluster_centers_
centers_unscaled = pd.DataFrame(scale.inverse_transform(centers), columns=X.columns)
centers_unscaled.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV,3P
0,10.603252,7.169919,1.571545,0.618699,1.017073,1.360976,0.326016
1,14.584946,4.155914,2.552688,0.793011,0.298925,1.551613,1.905376
2,24.703175,6.147619,6.538095,1.423810,0.557143,3.358730,2.479365
3,6.930968,3.385806,1.602581,0.588387,0.338710,0.903226,0.727742
4,18.003030,10.528788,2.457576,0.963636,1.662121,2.187879,0.533333
5,10.950000,4.895283,2.076415,1.259434,0.526415,1.311321,1.233962
6,15.647619,4.553333,5.851429,1.321905,0.400952,2.478095,1.272381


<IPython.core.display.Javascript object>

In [10]:
# GM7
gmm = GaussianMixture(n_components=7)
gmm.fit(X_scale)
means = gmm.means_
means_unscaled = pd.DataFrame(scale.inverse_transform(means), columns=X.columns)
means_unscaled.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV,3P
0,13.140305,5.163436,1.818691,0.922527,0.374380,1.291887,1.593076
1,16.696703,9.614633,3.121460,0.939816,1.375481,2.231703,0.651979
2,6.418082,3.378639,1.069338,0.558752,0.378802,0.767249,0.782258
3,13.807634,3.054768,3.823278,0.922594,0.240078,1.825375,1.632634
4,15.539082,4.875606,4.485254,1.325292,0.470377,2.167170,1.447210
5,24.240552,6.680634,7.645604,1.467970,0.546856,3.559169,2.150844
6,10.004398,6.564946,1.428395,0.698595,0.976518,1.265594,0.423514


<IPython.core.display.Javascript object>

In [11]:
# 3 cluster GM model produced very similar architypes to the KMeans model.
# The 7 cluster models pruduced very different clusters from each other but it's not
# clear if all the clusters represent a meningful architype

<IPython.core.display.Javascript object>

Predict the likelihood of each player belonging to one of the 3 clusters using the GMM model. Then, calculate the entropy for each set of predicted probabilities. 

We will use entropy as a measure of how confident we are in the predicted class label. If we had no confidence in our prediction, we would assign 33% probability to each class, while if we were totally confident, we would assign 100% to one class. Entropy would be at a maximum in the "no confidence" scenario and a minimum in the "full confidence" scenario, which makes it a reasonable way to quantify our uncertainty in our prediction. There are certainly other methods as well; feel free to experiment with them if desired.

Which five predicted labels are we least confident about? Which five are we most confident about? Print out the associated details (season, player name, stats, etc.) from those players.

In [17]:
# answer goes here
gmm = GaussianMixture(n_components=3)
gmm.fit(X_scale)
probs = gmm.predict_proba(X_scale)


entropy = entr(probs)

entorpy_df = pd.DataFrame(entropy, columns=["cluster_0", "cluster_1", "cluster_2"])

nba_df2["cluster"] = entorpy_df.apply(lambda x: x.idxmin(), axis=1)
nba_df2["entropy"] = entorpy_df.apply(lambda x: x.min(), axis=1)
nba_df2.sort_values(by="entropy").head(5)

,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,cluster,entropy
797,2018,DeAndre Jordan,C,29,LAC,77,77,31.5,4.8,7.5,...,10.9,15.2,1.5,0.5,0.9,1.8,2.6,12.0,cluster_0,-0.000000e+00
182,2019,Paul George,SF,28,OKC,77,77,36.9,9.2,21.0,...,6.8,8.2,4.1,2.2,0.4,2.7,2.8,28.0,cluster_0,-0.000000e+00
767,2018,Brandon Ingram,SF,20,LAL,59,59,33.5,6.1,12.9,...,4.4,5.3,3.9,0.8,0.7,2.5,2.8,16.1,cluster_2,9.575787e-89
192,2019,Treveon Graham,SG,25,BRK,35,21,20.4,1.8,5.5,...,2.4,3.1,1.0,0.4,0.2,0.5,1.9,5.3,cluster_2,1.956799e-60
389,2019,Kelly Olynyk,PF,27,MIA,79,36,22.9,3.3,7.1,...,3.8,4.7,1.8,0.7,0.5,1.4,2.3,10.0,cluster_2,2.021111e-49


<IPython.core.display.Javascript object>